In [ ]:
# Alex Chansky
# 2/19/21, 2/20/21
# Purpose: Create new folder of quake name, extract DEM and place there and project. 
    # Then, go into Downloads directly and use GDAL codes to create slope, TPI, TRI
    # Then Use the last block of this code to project slope, TPI, and TRI into appropriate folders

In [18]:
# Import system modules
import pandas as pd 
import os
import arcpy
from arcpy.sa import *

In [19]:
arcpy.env.overwriteOutput = True

In [20]:
from pathlib import Path
downloads_path = str(Path.home() / "Downloads")

In [21]:
# List of quakes and associated projections
eq_list = pd.read_csv("R:\Liquefaction_RemoteSensing\Projects\GeospatialModel\global_data_2020\eq_projections.csv")

In [22]:
# Locations of PGA and PGV
input_path = "R:\Liquefaction_RemoteSensing\Projects\GeospatialModel\global_data_2020\PGA_PGV"

In [6]:
# Identify DEMs and associated standard deviations
dem_mosaic = downloads_path + "\\DEM_Mosaic.tif"
dem_std_mosaic = downloads_path + "\\DEM_Mosaic_STDs.tif"

In [23]:
# Number of events
num_eq = eq_list.shape[0]

In [24]:
# Making sure we use smaller cell size (for next group (mosaic has smaller cell size than PGA))
arcpy.env.cellSize = "MINOF"

In [25]:
sr = arcpy.SpatialReference(4326)

In [12]:
for x in range(17,num_eq):
    eq_name = eq_list['NewFolder'][x]
    eq_proj = eq_list['ProjectionCode'][x]
    pga = input_path + "\\" + eq_name + "\\pga_mean.flt"
    
    # Create folder
    out_folder_path = "R:\Liquefaction_RemoteSensing\Projects\GeospatialModel\local_data_2021_Alex" 
    arcpy.CreateFolder_management(out_folder_path, eq_name)

    # Extract DEM, Define and project
    out_DEM = ExtractByMask(dem_mosaic, pga)
    out_DEM.save(out_folder_path+"\\"+eq_name+"\\"+eq_name+"_GMTED_clipped.tif")
    arcpy.DefineProjection_management(out_DEM, sr)
    out_DEM_proj = out_folder_path+"\\"+eq_name+"\\"+eq_name+"_GMTED_clipped_proj.tif"
    arcpy.management.ProjectRaster(out_DEM, out_DEM_proj, int(eq_proj),"BILINEAR")
    
    
    # Extract DEM std dev, define and project
    out_DEM_std = ExtractByMask(dem_std_mosaic, pga)
    out_DEM_std.save(out_folder_path+"\\"+eq_name+"\\"+eq_name+"_GMTED_std_clipped.tif")
    arcpy.DefineProjection_management(out_DEM_std, sr)
    out_DEM_std_proj = out_folder_path+"\\"+eq_name+"\\"+eq_name+"_GMTED_std_clipped_proj.tif"
    arcpy.management.ProjectRaster(out_DEM_std, out_DEM_std_proj, int(eq_proj),"BILINEAR")

In [39]:
# Projecting TRI, TPI, slope WITHOUT creating new files
for x in range(1,num_eq):
    eq_name = eq_list['NewFolder'][x]
    eq_proj = eq_list['ProjectionCode'][x]
    
    out_folder_path = "R:\Liquefaction_RemoteSensing\Projects\GeospatialModel\local_data_2021_Alex"
    
    slope = out_folder_path + "\\"+eq_name+"\\"+eq_name+"_slope.tif"
    slope_proj = out_folder_path + "\\"+eq_name+"\\"+eq_name+"_slope_proj.tif"
    
    arcpy.DefineProjection_management(slope, int(eq_proj))
    
    TRI = out_folder_path + "\\"+eq_name+"\\"+eq_name+"_TRI.tif"
    TRI_proj = out_folder_path + "\\"+eq_name+"\\"+eq_name+"_TRI_proj.tif"
    
    arcpy.DefineProjection_management(TRI, int(eq_proj))
    
    TPI = out_folder_path + "\\"+eq_name+"\\"+eq_name+"_TPI.tif"
    TPI_proj = out_folder_path + "\\"+eq_name+"\\"+eq_name+"_TPI_proj.tif"
    
    arcpy.DefineProjection_management(TPI, int(eq_proj))